### Objetivo: Implementar e testar o pipeline completo

Conteúdo:
- Implementação do pipeline hierárquico
- Função de predição end-to-end
- Teste com conjunto de dados reservado
- Análise da "acurácia em cascata"
- Análise de propagação de erros
- Métricas do sistema completo